In [1]:
%pylab inline 
import pandas as pd 
import json

Populating the interactive namespace from numpy and matplotlib


In [2]:
# csv files for answer responses
gdrive_path = "/Users/dorislee/Google Drive/Turn/user_study/evaluation_study/transcription_and_analysis/"
# user study logs
box_path = "/Users/dorislee/Box/viz-summarization-user-study-material/" 

In [3]:
import glob

In [4]:
df_data = []
for filename in glob.glob("study_logs/*log"):
    line_num=0
    with open(filename) as f:
        for line in f:
            lsp = line.split(',')
            date = lsp[0]
            user = lsp[1][1:-1]
            task = lsp[2][1:-1]
            dataset_name = lsp[3][1:-1]
            clicks = json.loads(",".join(lsp[4:]))
            if dataset_name!="default":
                dataset = json.loads(open("../system/static/generated_dashboards/"+dataset_name).read().replace("\\",""))
                for nodeID in dataset.keys():
                    df_data.append([line_num,date,dataset_name,user,task,nodeID,clicks[nodeID],dataset[nodeID][-1]["filter"],dataset[nodeID][-1]["populationSize"]])
            line_num +=1

In [5]:
# df[df["visualization"].str.contains("duration=30\+ min")&~df["visualization"].str.contains(",")]#&(df["interested"]="interested")]

In [6]:
df = pd.DataFrame(df_data,columns=["line_num","date","dataset_name","user","task","nodeID","clicks","visualization","populationSize"])
df["date"] = pd.to_datetime(df["date"],format="%Y-%m-%d-%H-%M-%S")

In [7]:
df.loc[df["clicks"]==0,"interested"]="unselected"
df.loc[df["clicks"]==1,"interested"]="unselected"
df.loc[df["clicks"]==2,"interested"]="interested"
df.loc[df["clicks"]==3,"interested"]="not interested"

In [8]:
#select the latest recorded rating (since all node are logged on the latest output, so we will just pick the latest one)
#df = df.loc[df.groupby(["user","task","nodeID"])[["date"]].idxmax()]
#select the latest logged output based on line numbering
df = df.loc[df.groupby(["user","task","nodeID"])["line_num"].idxmax()]

In [9]:
df = df[["user","visualization","interested","task"]]

In [10]:
def cleanFilterVal(x):
    if x=="#":
        return "overall"
    else:
        return x.replace("$","=").replace("#",", ")[1:-2]

In [11]:
df["visualization"]= df["visualization"].apply(cleanFilterVal)

In [12]:
df["user"] = df["user"].astype("int")

In [13]:
df = df[~df["task"].str.contains("T")] #ignore the titanic tutorial tasks

In [16]:
#make sure each user only does the task that they are assigned
user_task =  pd.read_csv(gdrive_path+"UserTaskAssignment.csv")

In [17]:
i =0 
df_data=[]
for row in df.iterrows():
    selected_user = user_task[user_task["User"]==row[1]["user"]]
    if row[1]["task"] in selected_user["Task1"].values[0] or row[1]["task"] in selected_user["Task2"].values[0]: 
        #print "pass"
        df_data.append(row[1][["user","visualization","interested","task"]])
    else:
        pass
        #print row[1]
    i +=1

In [18]:
df_data =pd.DataFrame(df_data)

In [35]:
assert (df_data.groupby("task").count() == 60).sum().sum()==18

- Had to manually clean up user_log_03_22.log because the second trial of user 11 was recorded as user 1, user 12 was also recorded as user 1
- Had to manually clean up user_log_03_18.log because user 8 was not recorded
- Had to manually clean up user_log_03_18.log because user  18 was recorded as user 1 for the second trial and there were A1 stuff mixed in there

cross checking that the results are same as what we expect

In [47]:
df.to_csv("user_click_data.csv",index=None)